## Introduction
Greetings from the Kaggle bot! This is an automatically-generated kernel with starter code demonstrating how to read in the data and begin exploring. If you're inspired to dig deeper, click the blue "Fork Notebook" button at the top of this kernel to begin editing.

## Exploratory Analysis
To begin this exploratory analysis, first import libraries and define functions for plotting the data using `matplotlib`. Depending on the data, not all plots will be made. (Hey, I'm just a simple kerneling bot, not a Kaggle Competitions Grandmaster!)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


There is 1 csv file in the current version of the dataset:


In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


The next hidden code cells define functions for plotting data. Click on the "Code" button in the published kernel to reveal the hidden code.

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()


In [ ]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()


In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()


Now you're ready to read in the data and use the plotting functions to visualize the data.

### Let's check the data file: /kaggle/input/16PF/data.csv

In [ ]:
data = pd.read_csv("/kaggle/input/16PF/data.csv", sep="\t")

Let's take a quick look at what the data looks like:

In [ ]:
data.head()

In [ ]:
data.shape

## Dataset breakdown in factors and positive and negatively keyed questions

This personality test consists of 164 statements about the subject, for each indicate how accurate it is on the scale of (1) disagree (2) slightly disagree (3) niether agree nor disagree (4) slightly agree (5) agree.

Sources: codebook and [IPIP](https://ipip.ori.org/new16pfkey.htm)



In [ ]:
positive_questions = [ # positive questions adding to the trait.
    "A1", "A2", "A3", "A4", "A5", "A6", "A7",       # Factor A: WARMTH 
    "B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", # Factor B: REASONING (Intellect)
    "C1", "C2", "C3", "C4", "C5",                   # Factor C: EMOTIONAL STABILITY
    "D1", "D2", "D3", "D4", "D5", "D6",             # Factor D: DOMINANCE (Assertiveness)
    "E1", "E2", "E3", "E4", "E5", "E6",  # Factor F: GREGARIOUSNESS (Liveliness)
    "F1", "F2", "F3", "F4", "F5", # Factor G: DUTIFULNESS (Rule-Consciousness)
    "G1", "G2", "G3", "G4", "G5", # Factor H: FRIENDLINESS (Social boldness)
    "H1", "H2", "H3", "H4", "H5", "H6",  # Factor I: SENSITIVITY
    "I1", "I2", "I3", "I4", "I5", "I6",  # Factor L: DISTRUST (Vigilance)
    "J1", "J2", "J3", "I4", "I5", "I6",  # Factor L: DISTRUST (Vigilance)


]

negative_questions = [ # negative questions substracting to the trait
    "A8", "A9", "A10",                 # WARMTH
    "B9", "B10", "B11", "B12", "B13",  # REASONING
    "C6", "C7", "C8", "C9", "C10",     # EMOTIONAL STABILITY

]

Distribution graphs (histogram/bar graph) of sampled columns:

In [ ]:
plotPerColumnDistribution(data, 10, 5)

Correlation matrix:

In [0]:
plotCorrelationMatrix(data, 25)

Scatter and density plots:

In [0]:
plotScatterMatrix(data, 20, 10)

## Conclusion
This concludes your starter analysis! To go forward from here, click the blue "Fork Notebook" button at the top of this kernel. This will create a copy of the code and environment for you to edit. Delete, modify, and add code as you please. Happy Kaggling!